<a href="https://colab.research.google.com/github/rmd-davis/bysykkel/blob/main/bysykkel_adventure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bysykkel i Trondheim

Du skal være konsulent for en dag, og din første kunde er Trondheim Bysykkel (et samarbeid mellom Trondheim Kommune og Clear Channel Norway AS).

Trondheim Kommune ønsker å tilby elsykler (i stedet for vanlige sykler) på de strekningene hvor folk bruker mest tid fra A til B, men de vet ikke hvilke strekgninger det gjelder. De ønsker denne informasjonen i både tabulær format og visualisert i et kart. 

De har gitt deg et Excel ark med ~140,000 turer fra 2021, og regner med at du klarer det på noen dager. Heldigvis kan du litt Pandas, og vet at du får til dette mye raskere.

Pandas docs: https://pandas.pydata.org/docs/

## Setup

In [ ]:
# Install packages needed for tutorial
# Restart runtime after running this cell

#!pip uninstall pandas-profiling
!pip install pandas-profiling==3.1.0 -q --no-warn-conflicts
!pip install pydeck -q --no-warn-conflicts

In [ ]:
# Clone the bysykkel github repo into colab
!git clone https://github.com/rmd-davis/bysykkel

In [ ]:
# Change directory into the bysykkel repository
%cd bysykkel

In [ ]:
# Load required packages

# Pandas for handling data
import pandas as pd

# Pandas profiling for automated exploratory data analysis
from pandas_profiling import ProfileReport

# Pydeck for mapping
import pydeck as pdk

# Matplotlib and pyplot for plotting

from matplotlib import pyplot as plt 
import seaborn as sns

# Import script for code testing
# This is only needed to test the codeblocks you write in this tutorial 
from functions import codetests

## Last inn data

In [ ]:
# Load data
df = pd.read_excel('data/bysykkel_2021.xlsx')

In [ ]:
'''
Inspect the data we loaded using df.head().
Note that the default setting for df.head() is to return the first 5 rows.
By specifying df.head(10), we get the first 10 rows instead.
'''

df.head(10)

In [ ]:
'''
Looks like we have some cleaning up to do!
In the Excel file, there is some other information at the top before the data starts on the 7th row.
Load data using pd.read_excel, but skip the first 6 rows and set the header row using the header parameter
'''

### Enter your code below this line



### Enter your code above this line


### Code test
codetest_01_input = df.iloc[0,0]
codetests.codetest_01_header(codetest_01_input)


In [ ]:
'''
Inspect data after adding the header parameter to the load function. 
If your code passed codetest_01, the first row should contain a trip from Station 118 to Station 108
'''

df.head()

## Automatic EDA (Exploratory Data Analysis)

Now that we have the dataset loaded, we can use Pandas Profiling to get some quick insights into the dataset.

https://pypi.org/project/pandas-profiling/

In [ ]:
# Generate Pandas Profiling report
profile = ProfileReport(df, title='Bysykkel Trondheim', html={'style':{'full_width':True}})
profile.to_notebook_iframe()

In [ ]:
'''
Use df.info() to get basic information about the dataset. 
Notice the Dtype for 'started_at' and 'ended_at'.
Also, notice the number of non-null values in 'ended_at', 'start_station_description', and 'end_station_description'
'''

df.info()

## Data cleanup

Issues with the dataset:

* 'started_at' and 'ended_at' are strings and not datetime variables
* Missing 'start_station_name'
* There are some extra columns we don't need
* There are duplicate rows
* Missing duration for each trip
* 'ended_at' has 703 missing values

### Dates

'started_at' and 'ended_at' are 'objects', but should be 'datetime'
<br>https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [ ]:
''' 
Convert 'started_at' and 'ended_at' to datetime variables.
Use pd.to_datetime()
To apply a transformation to a particular column, you can use the convention below:
df['column_name'] = transformation(df['column_name'])

'''

### Enter your code below this line



### Enter your code above this line


### Code test
codetest_02_input = df.dtypes
codetests.codetest_02_dtypes(codetest_02_input)

In [ ]:
'''
Use df.info() again, and now notice the Dtype for 'started_at' and 'ended_at'.
If you passed the codetest above, they will now both be datetime variables.
'''

df.info()

### Missing data
We are missing the 'start_station_name' column to match the 'end_station_name' column.
<br>Luckily, we have another Excel file that lists station IDs and station names. We can combine the information from both files using df.merge()
<br>https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html 

In [ ]:
# Load 'start_station_id.xlsx' into a new datadrame called 'station_names'

station_names = pd.read_excel('data/start_station_id.xlsx')

In [ ]:
# Inspect station_names dataframe
station_names

In [ ]:
'''
Now we will merge the original dataframe and the information in the station_names dataframe.
Use df.merge, merge on 'start_station_id', and use a 'left' merge.
'''

### Enter your code below this line



### Enter your code above this line


### Code test
codetest_03_input = df
codetests.codetest_03_merge(codetest_03_input)

### Delete unnecessary columns

Delete 'start_station_description' and 'end_station_description'. 
<br>Use df.drop( ).
<br> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html


In [ ]:
'''
Delete columns using df.drop()
You can either assign the result of df.drop to df, or use the inplace parameter
'''

### Enter your code below this line



### Enter your code above this line


### Code test
codetest_04_input = df
codetests.codetest_04_drop(codetest_04_input)

### Remove duplicates

In [ ]:
# First, we find the duplicated rows using df.duplicated()

df[df.duplicated(keep=False)]

In [ ]:
'''
Now we will use df.drop() to drop a single row from the dataframe.
We will drop the row with index = 29143.
Note that you can specify which axis to drop, but it is not necessary since the default behaviour is axis=0, which is what we want to drop a row.
'''

### Enter your code below this line



### Enter your code above this line


### Code test
codetest_05_input = df
codetests.codetest_05_duplicates(codetest_05_input)

### Add a column with trip duration
Add a column which contains the duration of each trip in seconds.
<br> Since we have 'started_at' and 'ended_at', Pandas can do the calculation for us automatically as a timedelta. 
<br>We'll use <code>df[column].dt.seconds</code> to get seconds as a float.

<br> Instead of you coding this one, we will just show you the power and time savings of using a vectorized approach to calculations. We'll use the <code>%%time</code> command to time different ways of performing the calculation.

In [ ]:
# Calculating duration by looping through each row iteratively.
# After each iteration, we'll build up a list of duration values
# After looping through all rows, we add the list of duration values to the dataframe

%%time

duration_list = []
duration_value = 0

for row in df.index:
    duration_value = (df.loc[row]['ended_at'] - df.loc[row]['started_at']).seconds
    duration_list.append(duration_value)

df['duration_loop'] = duration_list

In [ ]:
# Calculating duration using a lambda function
# Here we build the duration column as we go

%%time

df['duration_lambda'] = df.apply(lambda x: (x['ended_at'] - x['started_at']).seconds, axis=1)

In [ ]:
# Finally, we'll look at a vectorized approach
%%time

# Calculate timedelta between 'ended_at' and 'started_at'
df['duration'] = df['ended_at'] - df['started_at']

# Convert column from timedelta to float of seconds
df['duration'] = df['duration'].dt.seconds

### Fix missing values in 'ended_at'
* There are many ways of fixing missing values
* The most important thing is to document what you are changing in your dataset, and how you are changing it.
* We will fill the missing values in 'duration' with the average trip duration, generate a column of timedeltas, and then use that to fix all ended_at values.

In [ ]:
# Fill missing values in the 'duration' column with the mean of all duration values
df['duration'].fillna(df['duration'].mean(), inplace=True)

# Create a timedelta column based on the duration column
# Timedeltas are used to make calculations based on variables in a datetime format, so we need to convert our duration column which is stored as a float
df['duration_timedelta'] = pd.to_timedelta(df['duration'], unit='s')

# Update all 'ended_at' values
df['ended_at'] = df['started_at'] + df['duration_timedelta']

# Drop the columns we no longer need in the dataframe
df = df.drop(columns = ['duration_timedelta','duration_loop','duration_lambda'])


## Check the result of the data cleanup

In [ ]:
df.info()

In [ ]:
# Use df.describe() or df.describe().transpose()
df.describe()

## Plot the longest trip

In [ ]:
# Select the n longest trips

# Set the number of trips you want to look at.
# Plot just the 1 longest trip first. Then change it to the 20 longest trips.
n_trips = 1

# Create a new dataframe with only the values we want to plot
df_map_data = df.sort_values(by='duration', na_position='first').tail(n_trips)

# Add columns for start coordinates and end coordinates in the format [long,lat]
df_map_data['start_coord'] = df_map_data.apply(lambda x: [x['start_station_longitude'],x['start_station_latitude']],axis=1)
df_map_data['end_coord'] = df_map_data.apply(lambda x: [x['end_station_longitude'],x['end_station_latitude']], axis=1)

# Generate the map
layer = pdk.Layer("LineLayer", df_map_data, get_source_position="start_coord",get_target_position="end_coord", get_color = [255,0,0], get_width = 3)
init_view_state = pdk.ViewState(longitude=10.3985, latitude=63.4256, zoom=12)
r = pdk.Deck(layers=layer, initial_view_state=init_view_state, map_style='light')
r.to_html();

# Appendix

# Other useful Pandas functions

https://pandas.pydata.org/docs/

The fastest way to solve a problem is usually through Google or StackOverflow.If you have a question, it's highly likely that it has already been asked and solved on StackOverflow.

There are TONS of articles with 'Top n Pandas Functions you must know'. Here's a selection of some of the functions we have use of in projects.

Remember that a lot of operations that can be done on a whole dataframe can also be done on a single column as a Series.

```
df.method() # Accesses entire Pandas DataFrame
df['column_name'].method() # Accesses only the column as a Pandas Series
```



---



- style (precision, conditional formatting, etc)
- pd.set_option() - https://pandas.pydata.org/docs/user_guide/options.html?highlight=options%20display
- df.nlargest og df.nsmallest - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nlargest.html?highlight=nlargest#pandas.DataFrame.nlargest
- df.idxmax() og df.idxmin() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html?highlight=idxmax#pandas.DataFrame.idxmax
- df.count() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html?highlight=count#pandas.DataFrame.count
- df.value_counts(dropna = False) - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html?highlight=value_counts#pandas.DataFrame.value_counts
- df.at_time og df.between_time - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at_time.html?highlight=at_time
- df.replace() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html?highlight=replace#pandas.DataFrame.replace
- df.fillna() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html?highlight=fillna#pandas.DataFrame.fillna
- df.groupby() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html?highlight=groupby#pandas.DataFrame.groupby
- df.query() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html?highlight=query#pandas.DataFrame.query
- df.where() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.where.html?highlight=where#pandas.DataFrame.where
- df['column_name'].isin() - https://pandas.pydata.org/docs/reference/api/pandas.Series.isin.html?highlight=isin#pandas.Series.isin
- df['column_name'].unique() - https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html?highlight=unique#pandas.Series.unique
- df['column_name'].nunique() - https://pandas.pydata.org/docs/reference/api/pandas.Series.nunique.html?highlight=nunique#pandas.Series.nunique
- df['column_name'].cumsum() - https://pandas.pydata.org/docs/reference/api/pandas.Series.cumsum.html?highlight=cumsum#pandas.Series.cumsum
- df['column_name'].pct_change() - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html?highlight=pct%20change#pandas.DataFrame.pct_change
- df['column_name'].shift() - https://pandas.pydata.org/docs/reference/api/pandas.Series.shift.html?highlight=shift#pandas.Series.shift
- Windowing operations - https://pandas.pydata.org/docs/user_guide/window.html#window-expanding


- df.pipe - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html?highlight=pipe#pandas.DataFrame.pipe 
- Using .pkl to compress CSV files - https://pandas.pydata.org/docs/reference/api/pandas.read_pickle.html 
- Handling larger amounts of data - https://pandas.pydata.org/docs/user_guide/scale.html
- Pandas Cookbook - https://pandas.pydata.org/docs/user_guide/cookbook.html
- Built-in plotting - https://pandas.pydata.org/docs/user_guide/visualization.html






## Bonus: Alternate strategy for filling missing durations
Using averages for each segment to fill values for those segments.

Note that this will not produce a result now since the code in '*Fix missing values in 'ended_at' '* has already cleaned up all the missing values in the 'ended_at' column.
BUT, all of this code below could replace the code in *'Fix missing values in 'ended_at''*

In [ ]:
# Make unique segment ids based on start station and end station
df['segment'] = df['start_station_id'].astype(str) + '_' + df['end_station_id'].astype(str)

unique_segments = df['segment'].unique()

print(f'Total number of unique segments: {len(unique_segments)}')

In [ ]:
# Create dictionary of segment names and mean duration for each segment
segment_means = {}

# Loop over all of the unique segments and calculate the mean duration for that segment based on all trips for that segment

for segment in unique_segments:
    temp_frame = df[df['segment'] == segment]
    segment_means[segment] = temp_frame['duration'].mean()

In [ ]:
# Extract dataframe with missing ended_at values
# Use df.isna()

missing_endtime = df[df['ended_at'].isna()]
missing_endtime = missing_endtime[['started_at','segment']]

# Show missing_endtime dataframe
missing_endtime

In [ ]:
# Fill in missing durations with values from segment_means dictionary

missing_endtime['duration'] = missing_endtime['segment'].apply(lambda x: segment_means[x]) 

# Create timedelta, needed for calculation of ended_at
missing_endtime['duration_dt'] = pd.to_timedelta(missing_endtime['duration'], unit='s')

# Calculate ended_at
missing_endtime['ended_at'] = missing_endtime['started_at'] + missing_endtime['duration_dt']

# Extract necessary columns, needed to fill original dataframe
missing_endtime = missing_endtime[['ended_at','duration']]

# Show updated dataframe
missing_endtime


# # Fill missing values in original dataframe based on values in missing_endtime dataframe
# # df.fillna() will automatically match based on index to fill in 'ended_at' and 'duration' values

# df = df.fillna(missing_endtime)